<a href="https://colab.research.google.com/github/Bongo-Seakhoa/Webscrape_project_1/blob/main/project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Webscraping sample project based on a real Upwork paid project.

The task given is the following : 
Write a Python algorithm that finds websites with the format `https://12695.portal.athenahealth.com/` and runs through the different variations of that and then pulls the practice name on the site into an Excel file.

Every site has the same format "welcome to the _ patient portal"- so we would just need the algorithm to run through all the different available variations of the URL format and then pull the URL and the portal and the "welcome to the _ patient portal" part of the site into an excel file.

In [1]:
#importing libraries
import requests 
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#base url
url = "https://12695.portal.athenahealth.com/"

In [3]:
response = requests.get(url)

In [4]:
#checking response
response

<Response [200]>

In [5]:
#Assigning the text to a html variable name 
html = response.text

In [6]:
soup = BeautifulSoup(html, 'html.parser')

In [7]:
soup.find_all('h1')[-1]

<h1 align="center">
Dr. Richard Handelsman</h1>

Looking at the website we can see that the last element in the 'h1' tag is always the practice name consistently in more than 4 website vaiations 12695-12699 and this then simplifies things for us as we can just extract that instead of the "welcome to the _ patient portal"

In [8]:
clinic_name = soup.find_all('h1')[-1].text.strip()

In [9]:
#Defining a function that will return the clinic name 

def get_clinic_name(clinic_id):
  url = f"https://{clinic_id}.portal.athenahealth.com/"
  response = requests.get(url)
  html = response.text
  soup = BeautifulSoup(html, 'html.parser')
  clinic_name = soup.find_all('h1')[-1].text.strip()
  return clinic_name

Now we do not know the range of the clinic ID's so that would be the next step. Upon inspection of the websites we find that there are two distinct errors that we get when a practice is not found 
- "Sorry, we can't find that practice. Make sure you typed the right address."
- "Payment Confirmation"

In [10]:
get_clinic_name(5000)

'Payment Confirmation'

Now we are going to combine it all and produce the final CSV document as requested by the client

In [11]:
#Defining start and end paramiters of the clinic ID's (website vairiations)
start = 12500
end = 12700

In [13]:
#Creating a master list to hold all our dictionaries 
master_list = []
#for loop to run through all our variations of the website as defined by our start and end.
for clinic_id in range (start,end+1):
  data_dict = {} 
  data_dict["clinic_id"] = clinic_id
  data_dict["clinic_name"] = get_clinic_name(clinic_id)
  if data_dict['clinic_name'] != "Payment Confirmation" and data_dict['clinic_name'] != "Sorry, we can't find that practice. Make sure you typed the right address." and data_dict['clinic_name'] != "":
    master_list.append(data_dict)

In [14]:
#Creating a dataframe out of the master_list list of dictionaries
df = pd.DataFrame(master_list)

In [15]:
#Creating the final CSV
df.to_csv('clinic_data.csv', index=False)